# Query `apiso:ServiceType` 

In [1]:
from owslib.csw import CatalogueServiceWeb
from owslib import fes
import numpy as np

In [13]:
#endpoint = 'https://dev-catalog.ioos.us/csw' 
#endpoint = 'http://gamone.whoi.edu/csw'
endpoint = 'https://data.ioos.us/csw'
#endpoint = 'https://ngdc.noaa.gov/geoportal/csw'
csw = CatalogueServiceWeb(endpoint,timeout=60)
print csw.version

2.0.2


In [14]:
csw.get_operation_by_name('GetRecords').constraints

[Constraint: SupportedISOQueryables - ['apiso:DistanceValue', 'apiso:Abstract', 'apiso:RevisionDate', 'apiso:Subject', 'apiso:KeywordType', 'apiso:Title', 'apiso:CRS', 'apiso:PublicationDate', 'apiso:Type', 'apiso:AlternateTitle', 'apiso:BoundingBox', 'apiso:AnyText', 'apiso:ParentIdentifier', 'apiso:Modified', 'apiso:Operation', 'apiso:Format', 'apiso:TempExtent_end', 'apiso:DistanceUOM', 'apiso:OrganisationName', 'apiso:ServiceType', 'apiso:TempExtent_begin', 'apiso:ResourceLanguage', 'apiso:ServiceTypeVersion', 'apiso:OperatesOn', 'apiso:Denominator', 'apiso:HasSecurityConstraints', 'apiso:OperatesOnIdentifier', 'apiso:GeographicDescriptionCode', 'apiso:Language', 'apiso:Identifier', 'apiso:OperatesOnName', 'apiso:TopicCategory', 'apiso:CreationDate', 'apiso:CouplingType'],
 Constraint: AdditionalQueryables - ['apiso:Lineage', 'apiso:Classification', 'apiso:Creator', 'apiso:Relation', 'apiso:OtherConstraints', 'apiso:SpecificationTitle', 'apiso:ResponsiblePartyRole', 'apiso:Specific

Search first for records containing the two text strings

In [15]:
val = 'COAWST'
filter1 = fes.PropertyIsLike(propertyname='apiso:AnyText',literal=('*%s*' % val),
                        escapeChar='\\',wildCard='*',singleChar='?')
filter_list = [ filter1 ]

In [16]:
val = 'USGS'
filter2 = fes.PropertyIsLike(propertyname='apiso:AnyText',literal=('*%s*' % val),
                        escapeChar='\\',wildCard='*',singleChar='?')
filter_list = [fes.And([filter1, filter2])] 

In [17]:
endpoint = 'http://gamone.whoi.edu/csw'  

csw = CatalogueServiceWeb(endpoint,timeout=60,version="2.0.2")
print csw.version
csw.getrecords2(constraints=filter_list,maxrecords=100,esn='full')
print len(csw.records.keys())
for rec in list(csw.records.keys()):
    print csw.records[rec].title 

2.0.2
4
COAWST Hindcast:Barnegat Bay:ADCIRC tides, real rivers, plume, lowpass Espresso bdry, NAM, new bathy
COAWST Forecast System : USGS : US East Coast and Gulf of Mexico (Experimental)
Hudson River and Jamaica Bay Model for Hurricane Sandy
COAWST Hindcast:MVCO/CBlast 2007:ripples with SWAN-40m res


In [18]:
endpoint = 'http://gamone.whoi.edu/csw/' 

csw = CatalogueServiceWeb(endpoint,timeout=60,version="2.0.2")
print csw.version
csw.getrecords2(constraints=filter_list,maxrecords=100,esn='full')
print len(csw.records.keys())
for rec in list(csw.records.keys()):
    print csw.records[rec].title 

2.0.2
4
COAWST Hindcast:Barnegat Bay:ADCIRC tides, real rivers, plume, lowpass Espresso bdry, NAM, new bathy
COAWST Forecast System : USGS : US East Coast and Gulf of Mexico (Experimental)
Hudson River and Jamaica Bay Model for Hurricane Sandy
COAWST Hindcast:MVCO/CBlast 2007:ripples with SWAN-40m res


In [19]:
val = 'G1SST'
filter1 = fes.PropertyIsLike(propertyname='apiso:AnyText',literal=('*%s*' % val),
                        escapeChar='\\',wildCard='*',singleChar='?')
filter_list = [ filter1 ]

In [20]:
val = 'GHRSST'
filter2 = fes.PropertyIsLike(propertyname='apiso:AnyText',literal=('*%s*' % val),
                        escapeChar='\\',wildCard='*',singleChar='?')
filter_list = [fes.And([filter1, filter2])] 

In [21]:
endpoint = 'https://ngdc.noaa.gov/geoportal/csw'      

csw = CatalogueServiceWeb(endpoint,timeout=60)
print csw.version
csw.getrecords2(constraints=filter_list,maxrecords=100,esn='full')
print len(csw.records.keys())
for rec in list(csw.records.keys()):
    print csw.records[rec].title 

2.0.2
2
G1SST, 1km blended SST
G1SST, 1km blended SST


In [22]:
endpoint = 'https://dev-catalog.ioos.us/csw'      

csw = CatalogueServiceWeb(endpoint,timeout=60)
print csw.version
csw.getrecords2(constraints=filter_list,maxrecords=100,esn='full')
print len(csw.records.keys())
for rec in list(csw.records.keys()):
    print csw.records[rec].title 

2.0.2
2
G1SST, 1km blended SST
G1SST, 1km blended SST


In [23]:
def fes_date_filter(start, stop, constraint='overlaps'):
    """
    Take datetime-like objects and returns a fes filter for date range
    (begin and end inclusive).
    NOTE: Truncates the minutes!!!
    Examples
    --------
    >>> from datetime import datetime, timedelta
    >>> stop = datetime(2010, 1, 1, 12, 30, 59).replace(tzinfo=pytz.utc)
    >>> start = stop - timedelta(days=7)
    >>> begin, end = fes_date_filter(start, stop, constraint='overlaps')
    >>> begin.literal, end.literal
    ('2010-01-01 12:00', '2009-12-25 12:00')
    >>> begin.propertyoperator, end.propertyoperator
    ('ogc:PropertyIsLessThanOrEqualTo', 'ogc:PropertyIsGreaterThanOrEqualTo')
    >>> begin, end = fes_date_filter(start, stop, constraint='within')
    >>> begin.literal, end.literal
    ('2009-12-25 12:00', '2010-01-01 12:00')
    >>> begin.propertyoperator, end.propertyoperator
    ('ogc:PropertyIsGreaterThanOrEqualTo', 'ogc:PropertyIsLessThanOrEqualTo')
    """
    start = start.strftime('%Y-%m-%d %H:00')
    stop = stop.strftime('%Y-%m-%d %H:00')
    if constraint == 'overlaps':
        propertyname = 'apiso:TempExtent_begin'
        begin = fes.PropertyIsLessThanOrEqualTo(propertyname=propertyname,
                                                literal=stop)
        propertyname = 'apiso:TempExtent_end'
        end = fes.PropertyIsGreaterThanOrEqualTo(propertyname=propertyname,
                                                 literal=start)
    elif constraint == 'within':
        propertyname = 'apiso:TempExtent_begin'
        begin = fes.PropertyIsGreaterThanOrEqualTo(propertyname=propertyname,
                                                   literal=start)
        propertyname = 'apiso:TempExtent_end'
        end = fes.PropertyIsLessThanOrEqualTo(propertyname=propertyname,
                                              literal=stop)
    else:
        raise NameError('Unrecognized constraint {}'.format(constraint))
    return begin, end

In [24]:
bbox = [-71.3, 42.03, -70.57, 42.63]
bbox_filter = fes.BBox(bbox,crs='urn:ogc:def:crs:OGC:1.3:CRS84')

from datetime import datetime, timedelta
import pytz
stop = datetime(2016, 8, 24, 0, 0, 0).replace(tzinfo=pytz.utc)
start = stop - timedelta(days=7)
begin, end = fes_date_filter(start, stop, constraint='overlaps')
print(start)
print(stop)
filter_list = [fes.And([filter1, filter2, bbox_filter, begin, end])] 
csw.getrecords2(constraints=filter_list, maxrecords=1000)
print len(csw.records.keys())
for rec in list(csw.records.keys()):
    print csw.records[rec].title 

2016-08-17 00:00:00+00:00
2016-08-24 00:00:00+00:00
1
G1SST, 1km blended SST


Now let's print out the references (service endpoints) to see what types of services are available

In [25]:
choice=np.random.choice(list(csw.records.keys()))
print(csw.records[choice].title)
csw.records[choice].references

G1SST, 1km blended SST


[{'scheme': 'WWW:LINK',
  'url': 'http://thredds.aoos.org/thredds/dodsC/G1_SST_GLOBAL.nc.html'},
 {'scheme': 'WWW:LINK',
  'url': 'http://www.ncdc.noaa.gov/oa/wct/wct-jnlp-beta.php?singlefile=http://thredds.aoos.org/thredds/dodsC/G1_SST_GLOBAL.nc'},
 {'scheme': 'OPeNDAP:OPeNDAP',
  'url': 'http://thredds.aoos.org/thredds/dodsC/G1_SST_GLOBAL.nc'},
 {'scheme': 'OGC:WCS',
  'url': 'http://thredds.aoos.org/thredds/wcs/G1_SST_GLOBAL.nc?service=WCS&version=1.0.0&request=GetCapabilities'},
 {'scheme': 'OGC:WMS',
  'url': 'http://pdx.axiomalaska.com/ncWMS/wms?service=WMS&version=1.3.0&request=GetCapabilities'},
 {'scheme': 'UNIDATA:NCSS',
  'url': 'http://thredds.aoos.org/thredds/ncss/G1_SST_GLOBAL.nc/dataset.html'}]

In [26]:
csw.records[choice].xml

'<csw:SummaryRecord xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:inspire_common="http://inspire.ec.europa.eu/schemas/common/1.0" xmlns:atom="http://www.w3.org/2005/Atom" xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:dct="http://purl.org/dc/terms/" xmlns:ows="http://www.opengis.net/ows" xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:gml="http://www.opengis.net/gml" xmlns:dif="http://gcmd.gsfc.nasa.gov/Aboutus/xml/dif/" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:gco="http://www.isotc211.org/2005/gco" xmlns:gmd="http://www.isotc211.org/2005/gmd" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:srv="http://www.isotc211.org/2005/srv" xmlns:ogc="http://www.opengis.net/ogc" xmlns:fgdc="http://www.opengis.net/cat/csw/csdgm" xmlns:inspire_ds="http://inspire.ec.europa.eu/schemas/inspire_ds/1.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:soape

We see that the `OPeNDAP` service is available, so let's see if we can add that to the query, returning only datasets that have text "COAWST" and "experimental" and that have an "opendap" service available. 

We should get the same number of records, as all COAWST records have OPeNDAP service endpoints.   If we get no records, something is wrong with the CSW server.

In [27]:
val = 'OPeNDAP'
filter3 = fes.PropertyIsLike(propertyname='apiso:ServiceType',literal=('*%s*' % val),
                        escapeChar='\\',wildCard='*',singleChar='?')
filter_list = [fes.And([filter1, filter2, filter3])]
csw.getrecords2(constraints=filter_list, maxrecords=1000)

In [28]:
print(len(csw.records.keys()))
for rec in list(csw.records.keys()):
    print('title:'+csw.records[rec].title) 
    print('identifier:'+csw.records[rec].identifier)
    print('modified:'+csw.records[rec].modified)
    print(' ')

2
title:G1SST, 1km blended SST
identifier:G1_SST_US_WEST_COAST
modified:2016-10-20
 
title:G1SST, 1km blended SST
identifier:G1_SST_GLOBAL
modified:2016-10-20
 
